
# Coherence metrics walkthrough

## Objectives
- Demonstrate how canonical coherence metrics extract C(t), mean ΔNFR, and mean ΔEPI from a running graph.
- Show the deterministic telemetry hook required by Phase-2 observability dashboards.
- Provide an assertion-friendly timeline that highlights the impact of successive glyph applications on coherence.

## Phase-2 dependencies
- [Phase-2 integration notes](../fase2_integration.md) — the coherence snapshots align with the observability contract described there.
- :mod:`tnfr.metrics.common` — Phase-2 dashboards reuse `compute_coherence` verbatim, so the primer doubles as living documentation.
- :mod:`tnfr.structural` — the operator sequencing mirrors the canonical emission→reception→coherence→resonance→transition segment executed by the runtime.

## Theoretical exposition
Coherence C(t) summarises how stable the network remains under the nodal equation. The accompanying ΔNFR and ΔEPI means trace how strongly the lattice is being reorganised. By recording the tuple `(C, mean ΔNFR, mean ΔEPI)` before and after deterministic glyph applications we verify that coherence gains correlate with controlled ΔNFR contributions. Because the hook writes explicit ΔNFR values, the averages match the scripted increments exactly, isolating the measurement contract used by the Phase-2 monitoring stack.

## Deterministic smoke check
The walkthrough spawns a two-node graph, applies the canonical emission→reception→coherence→resonance→transition sequence, and records coherence metrics after each glyph. The resulting timeline must show an increasing C(t) while ΔNFR contributions decay as the scripted iterators exhaust their increments. Any deviation would signal a regression in the coherence aggregator or in the ΔNFR ledger.


In [ ]:

from typing import Dict, Iterable, Tuple

from tnfr.constants import DNFR_PRIMARY, EPI_PRIMARY, VF_PRIMARY
from tnfr.dynamics import set_delta_nfr_hook
from tnfr.metrics.common import compute_coherence
from tnfr.structural import Coherence, Emission, Reception, Resonance, Transition, create_nfr, run_sequence

G, seed = create_nfr("coherence-seed", epi=0.36, vf=1.15, theta=0.05)
_, partner = create_nfr("coherence-partner", graph=G, epi=0.33, vf=1.05, theta=-0.02)
G.add_edge(seed, partner)

increments: Dict[str, Iterable[Tuple[float, float]]] = {
    seed: iter([(0.06, 0.02), (0.04, 0.01), (0.03, 0.0), (0.02, -0.005), (0.01, -0.005)]),
    partner: iter([(0.03, 0.0), (0.02, -0.005), (0.015, -0.005), (0.01, -0.005), (0.005, -0.005)]),
}


def snapshot(label: str) -> dict[str, float]:
    C, dnfr_mean, depi_mean = compute_coherence(G, return_means=True)
    return {
        "label": label,
        "C": round(C, 6),
        "ΔNFR_mean": round(dnfr_mean, 6),
        "ΔEPI_mean": round(depi_mean, 6),
    }


timeline = [snapshot("initial")]


def telemetry_hook(graph):
    for node_id, iterator in increments.items():
        dnfr, vf_shift = next(iterator, (0.0, 0.0))
        data = graph.nodes[node_id]
        vf = float(data[VF_PRIMARY])
        epi = float(data[EPI_PRIMARY])
        data[DNFR_PRIMARY] = dnfr
        data[EPI_PRIMARY] = epi + vf * dnfr
        data[VF_PRIMARY] = vf + vf_shift
    timeline.append(snapshot(f"step_{len(timeline)}"))


set_delta_nfr_hook(G, telemetry_hook, note="coherence metrics smoke")
run_sequence(G, seed, [Emission(), Reception(), Coherence(), Resonance(), Transition()])

timeline
